In [1]:
import nibabel as nib
import numpy as np

sphere = "data/tpl-fsLR_den-32k_hemi-L_sphere.surf.gii"
sphere = nib.load(sphere)
coordinates, vertices = sphere.agg_data()

x = "data/Monkey_GC1_nPDF_to_Human.L.32k_fs_LR.shape.gii"
x = nib.load(x).agg_data()
y = "data/Monkey_GC1_PDF_to_Human.L.32k_fs_LR.shape.gii"
y = nib.load(y).agg_data()

In [2]:
coordinates

array([[ 85.06508 ,   0.      ,  52.57311 ],
       [  0.      , -52.57311 ,  85.06508 ],
       [-85.06508 ,   0.      ,  52.57311 ],
       ...,
       [ -1.928835, -48.54365 , -87.40591 ],
       [ -3.834274, -49.721283, -86.67809 ],
       [ -1.921215, -50.295624, -86.409836]], dtype=float32)

In [3]:
v = coordinates / np.sqrt((coordinates ** 2).sum(axis=1))[:, np.newaxis]

In [4]:
r = np.zeros(v.shape[0])

vals = np.logical_and(x != 0, y != 0)
vals

array([ True,  True,  True, ...,  True,  True,  True])

In [12]:
v

array([[ 0.85065085,  0.        ,  0.52573115],
       [ 0.        , -0.52573115,  0.85065085],
       [-0.85065085,  0.        ,  0.52573115],
       ...,
       [-0.01928835, -0.48543656, -0.87405914],
       [-0.03834274, -0.49721283, -0.86678094],
       [-0.01921215, -0.5029562 , -0.86409837]], dtype=float32)

In [5]:
cos_angle = v @ v[0]
cos_angle

array([ 1.0000001 ,  0.44721365, -0.44721365, ..., -0.47592777,
       -0.48831   , -0.47062624], dtype=float32)

In [6]:
angle = np.arccos(np.clip(cos_angle, -1, 1))
angle

array([0.       , 1.1071486, 2.034444 , ..., 2.066815 , 2.0809484,
       2.0607967], dtype=float32)

In [7]:
j = (np.degrees(angle) < 30) & vals

r[j] = 1
# 将数据格式转换为int32
r = r.astype(np.int32)
new_gifti = nib.gifti.GiftiDataArray(r)
new_gifti = nib.gifti.GiftiImage(darrays=[new_gifti])
nib.save(new_gifti, "test.func.gii")

In [8]:
def surflocalcorr(x, y, sph, a=30, method="pearsonr"):
    if isinstance(x, str):
        x = nib.load(x)
    x = x.agg_data()
    if isinstance(y, str):
        y = nib.load(y)
    y = y.agg_data()
    if isinstance(sph, str):
        sph = nib.load(sph)
    coordinates, _ = sph.agg_data()
    v = coordinates / np.sqrt((coordinates ** 2).sum(axis=1))[:, np.newaxis]
    r = np.zeros(v.shape[0])

    # no values
    vals = np.logical_and(x != 0, y != 0)

    for i in range(v.shape[0]):
        cos_angle = v @ v[i]
        angle = np.arccos(np.clip(cos_angle, -1, 1))
        j = (np.degrees(angle) < a) & vals
        # Ensure enough values for stable correlation
        assert np.sum(j) > 30, "Not enough values for stable correlation, try increasing the angle"
        if method == "pearsonr":
            r[i] = np.corrcoef(x[j], y[j])[0, 1]
        elif method == "spearmanr":
            pass

    r[~vals] = 0
    return r

In [9]:
sphere = "data/tpl-fsLR_den-32k_hemi-L_sphere.surf.gii"
x = "data/Monkey_GC1_nPDF_to_Human.L.32k_fs_LR.shape.gii"
y = "data/Monkey_GC1_PDF_to_Human.L.32k_fs_LR.shape.gii"

corr = surflocalcorr(x, y, sphere, a=30, method="pearsonr")

In [11]:
from neuromaps.datasets import fetch_fslr
sphere_L, sphere_R = fetch_fslr(density='32k')['sphere']
corr = surflocalcorr(x, y, nib.load(sphere_L), a=30, method="pearsonr")

In [1]:
import os
import sys
project_root = "/Users/burgerhuang/Documents/ImageUtilisTools"
if project_root not in sys.path:
    sys.path.append(project_root)
from ImageUtilisTools.Stats.surf_local_correlation import surflocalcorr

sphere = "data/tpl-fsLR_den-32k_hemi-L_sphere.surf.gii"
x = "data/Monkey_GC1_nPDF_to_Human.L.32k_fs_LR.shape.gii"
y = "data/Monkey_GC1_PDF_to_Human.L.32k_fs_LR.shape.gii"

corr = surflocalcorr(x, y, sphere, a=30, method="spearmanr", return_gifti=True)
corr.to_filename("test_corr.func.gii")